In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable
from chainer import optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [2]:
# Set data

from sklearn import datasets
iris = datasets.load_iris()
X = iris.data.astype(np.float32)
Y = iris.target
N = Y.size
Y2 = np.zeros(3 * N).reshape(N,3).astype(np.float32)
for i in range(N):
    Y2[i,Y[i]] = 1.0

index = np.arange(N)
xtrain = X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0],:]
xtest = X[index[index % 2 == 0],:]
yans = Y[index[index % 2 == 0]]

In [3]:
# Define model

class IrisChain(Chain):
    def __init__(self):
        super(IrisChain, self).__init__(
            l1=L.Linear(4,6),
            l2=L.Linear(6,3),
        )

    def __call__(self,x,y):
        return F.mean_squared_error(self.fwd(x), y)

    def fwd(self,x):
         h1 = F.sigmoid(self.l1(x))
         h2 = self.l2(h1)
         return h2

In [17]:
# Initialize model

model = IrisChain()
optimizer = optimizers.SGD()
optimizer.setup(model)

In [18]:
# Learn

n = 75
bs = 25
for j in range(10000):
    accum_loss = None
    sffindx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = Variable(xtrain[sffindx[i:(i+bs) if (i+bs) < n else n]])
        y = Variable(ytrain[sffindx[i:(i+bs) if (i+bs) < n else n]])
        model.zerograds()
        loss = model(x,y)
        # 誤差の累積を求める
        accum_loss = loss if accum_loss is None else accum_loss + loss
        # loss.backward()
        # optimizer.update()
    accum_loss.backward()
    optimizer.update()

In [19]:
# Test

xt = Variable(xtest, volatile='on')
yy = model.fwd(xt)

ans = yy.data
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    cls = np.argmax(ans[i,:])
    print (ans[i,:], cls)
    if cls == yans[i]:
        ok += 1

print (ok, "/", nrow, " = ", (ok * 1.0)/nrow)
print("loss = " , accum_loss.data)

[ 1.01850522 -0.0324105   0.01266748] 0
[ 1.00474334 -0.00530291  0.00327746] 0
[ 1.01814461 -0.05223134  0.02181039] 0
[ 0.99902964 -0.03876644  0.01833571] 0
[ 0.96661794  0.08061761 -0.02583113] 0
[ 1.02540338 -0.04254195  0.0156991 ] 0
[ 0.99140793  0.07339874 -0.02618926] 0
[ 1.04493749 -0.11618403  0.04253659] 0
[ 1.03542852 -0.13077796  0.04818926] 0
[ 1.02229226 -0.03435537  0.01111672] 0
[ 0.99789572  0.04977974 -0.01820823] 0
[ 1.0207448  -0.13318032  0.056332  ] 0
[ 0.95068437  0.10062715 -0.02823707] 0
[  9.89619434e-01   1.81600451e-03   7.70874321e-04] 0
[ 1.01827788 -0.01273677  0.00377042] 0
[ 0.96918112  0.09006122 -0.02947678] 0
[ 1.02583551 -0.09643844  0.04197056] 0
[ 0.98948455  0.0764828  -0.02652695] 0
[ 1.03340364 -0.04015371  0.01186047] 0
[ 0.9858067   0.03036821 -0.00840046] 0
[ 1.02064967 -0.06639391  0.02483866] 0
[ 0.994995   -0.01271886  0.00919513] 0
[ 0.98472613 -0.01122972  0.00905133] 0
[ 1.01475751 -0.04911444  0.02187613] 0
[ 1.02295756 -0.04469693 

In [20]:
test_index = 36
xa = Variable(np.array([xtest[test_index]]).astype(np.float32()), volatile='on')
ya = model.fwd(xa)

print(ya.data)
print(yans[test_index])

[[-0.01772024  0.45251468  0.60276628]]
1
